### 아이디어
- 1번과 2번 쿼리에 대해서 다음과 같이 정리할 수 있다.
  - $q1(x, y, v): A_i := (A_i + v) \mod C$\
  리프 노드가 아닌 노드 $A'$ 에 대해서 \
  $A' = (A_x + v) \mod C + (A_{x+1} + v) \mod C + \cdots + (A_y + v) \mod C$ \
  $= (A_x + A_{x+1} + \cdots + A_y) \mod C + k \cdot v \mod C$
  - $q2(x, y, v) : A_i := (A_i \cdot v) \mod C$\
  $A' = (A_x \cdot v) \mod C + (A_{x+1} \cdot v) \mod C + \cdots + (A_y \cdot v) \mod C$\
  $= (A_x + A_{x+1} + \cdots + A_y) \cdot v \mod C$  
- 두 쿼리에 대한 방법이 다르다. 하나는 더해야 하고 하나는 곱해야 한다. 이를 어떻게 처리할 수 있을까? 내가 생각한 방법은 3가지 정도가 있다.
  1. st를 2개 만든다.
  2. 1번 쿼리 연산을 2번쿼리 연산처럼 변환하는 과정이 존재한다. 혹은 통일된 방법이 존재한다.
  3. 쿼리를 2개 만든다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

MOD = 10**9 + 7
class LazySegmentTree:
  def __init__(self, L, f, fl, fll, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(L)
    self.seg = [default] * self.len + L
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, idx, val) :
    self.seg[idx] = self.fl(self.seg[idx], val, self.cnt[idx])
    if idx < self.len :
      self.lazy[idx] = self.fll(self.lazy[idx], val)

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _build(self, idx) :
    while idx :
      if idx < self.len :
        self._apply(idx)
        self.seg[idx] = self.f(self.seg[idx*2], self.seg[idx*2+1])
      idx >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.seg[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  f = lambda a, b : (a + b) % MOD
  fl = lambda i, v, cnt : (i * v[0] + v[1] * cnt) % MOD
  fll = lambda x, y : ((x[0] * y[0]) % MOD, (x[1] * y[0] + y[1]) % MOD)
  lst = LazySegmentTree(L, f, fl, fll, ldefault=(1, 0))
  M = int(input())
  ans = []
  for _ in range(M) :
    q, *arg = map(int, input().split())
    if q == 1 :
      s, e, x = arg
      lst.update(s-1, e-1, (1, x))
    elif q == 2 :
      s, e, x = arg
      lst.update(s-1, e-1, (x, 0))
    elif q == 3 :
      s, e, x = arg
      lst.update(s-1, e-1, (0, x))
    elif q == 4 :
      s, e = arg
      ans.append(lst.query(s-1, e-1) % MOD)

  sys.stdout.write('\n'.join(map(str, ans)))

sol()

### 풀이
- lazy를 확장해서 단일 항이 아닌 tuple로 확장할 수 있다.\
이를 이용하여 lazy값 $(x, y)$ 에 대한 값 $v$ 에 대한 쿼리의 처리를 $x \cdot v + y$ 로 처리할 수 있다.
- 쿼리 연산이 원하고자 하는 바는 $\sum_{i=x}^y \mod \text c$ 이므로 `f = lambda x, y: (x + y) % c` 가 된다.
- f 는 add 연산을 하므로, 서브 노드가 $n$ 개인 노드 $A_i: v_i$ 에 대해서 $A_i$ 의 서브 노드를 $a_j, a_{j+1}, ..., a_{j+n-1}$ 라고 할 때, 연산 $(x, y)$ 에 대해 다음이 성립한다.
  - $A_i = \sum_{j=i}^{i+n-1}{v_j \cdot x + y = y \cdot n} + \sum_{j=i}^{i+n-1} v_j \cdot x$
  - 따라서 `fl = lambda i, (x, y), cnt: (i * x + y * cnt) % c` 가 된다.
- fll은 각 1, 2, 3 쿼리의 연산 정보를 lazy에 올바르게 보관 해야한다. 특히, 연산자 우선순위를 고려해야 한다.
  - 곱셈의 경우, 각 scaler를 곱해주면 된다.
  - 덧셈의 경우, 예를들어 덧셈 - 곱셈의 순서로 쿼리가 들어온다면, 처음 덧셈을 적용한 정보가 곱셈때에도 적용되어야 한다. 
  - 따라서 `fll = lambda (x1, x2), (y1, y2): ((x1 * y1) % c, (x2 * y1 + y2) % c)` 가 된다.